## How-to guide for Cloud Spend use-case on Abacus.AI platform
This notebook provides you with a hands on environment to build an cloud spend alert model using the Abacus.AI Python Client Library.

We'll be using the [Cloud Spend Dataset](https://s3.amazonaws.com//realityengines.exampledatasets/cloud_operations/cloud_spend.csv), which, as the name suggests, contains information about cloud spends.

1. Install the Abacus.AI library

In [ ]:
!pip install abacusai

We'll also import pandas and pprint tools for visualization in this notebook:

In [ ]:
import pandas as pd # A tool we'll use to download and preview CSV files
import pprint # A tool to pretty print dictionary outputs
pp = pprint.PrettyPrinter(indent=2)

2. Add your Abacus.AI [API Key](https://abacus.ai/app/profile/apikey) generated using the API dashboard as follows:

In [ ]:
#@title Abacus.AI API Key

api_key = ''  #@param {type: "string"}

3. Import the Abacus.AI library and instantiate a client.

In [ ]:
from abacusai import ApiClient
client = ApiClient(api_key)

## 1. Create a Project

Abacus.AI projects are containers that have datasets and trained models. By specifying a business **Use Case**, Abacus.AI tailors the deep learning algorithms to produce the best performing model catered specifically for your data.

We'll call the `list_use_cases` method to retrieve a list of the Use Cases currently available on the Abacus.AI platform.

In [ ]:
client.list_use_cases()

In this notebook, we're going to create a cloud spend alert model using the Cloud Spend dataset. The 'OPERATIONS_CLOUD' use case is best tailored for this situation.

In [ ]:
#@title Abacus.AI Use Case

use_case = 'OPERATIONS_CLOUD'  #@param {type: "string"}

By calling the `describe_use_case_requirements` method we can view what datasets are required for this use_case.

In [ ]:
for requirement in client.describe_use_case_requirements(use_case):
  pp.pprint(requirement.to_dict())

Finally, let's create the project.

In [ ]:
cloud_project = client.create_project(name='Cloud Spend Project', use_case=use_case)
cloud_project.to_dict()

## 2. Add Datasets to your Project

Abacus.AI can read datasets directly from `AWS S3` or `Google Cloud Storage` buckets, otherwise you can also directly upload and store your datasets with Abacus.AI. For this notebook, we will have Abacus.AI read the datasets directly from a public S3 bucket's location.

We are using one dataset for this notebook. We'll tell Abacus.AI how the dataset should be used when creating it by tagging the dataset with a special Abacus.AI **Dataset Type**.
- [Cloud Spend Dataset](https://s3.amazonaws.com//realityengines.exampledatasets/cloud_operations/cloud_spend.csv) (**TIMESERIES**): 
This dataset contains information about a company's cloud expenses, including the start date, service, and log cost.

### Add the dataset to Abacus.AI

First we'll use Pandas to preview the file, then add it to Abacus.AI.

In [ ]:
pd.read_csv('https://s3.amazonaws.com//realityengines.exampledatasets/cloud_operations/cloud_spend.csv')

Using the Create Dataset API, we can tell Abacus.AI the public S3 URI of where to find the datasets. We will also give each dataset a Refresh Schedule, which tells Abacus.AI when it should refresh the dataset (take an updated/latest copy of the dataset).

If you're unfamiliar with Cron Syntax, Crontab Guru can help translate the syntax back into natural language: [https://crontab.guru/#0_12_\*_\*_\*](https://crontab.guru/#0_12_*_*_*)

**Note: This cron string will be evaluated in UTC time zone**

In [ ]:
cloud_dataset = client.create_dataset('Cloud Spend', 
                                     location='s3://realityengines.exampledatasets/cloud_operations/cloud_spend.csv',
                                     project_id=cloud_project.project_id, 
                                     dataset_type='TIMESERIES',
                                     refresh_schedule='0 12 * * *')
datasets = [cloud_dataset]

### Check dataset processing status and get dataset schemas when processing finishes

In [ ]:
for dataset in datasets:
    dataset.wait_for_inspection()
    print(f'{dataset.name} Schema:')
    pp.pprint(client.get_schema(cloud_project.project_id, dataset.dataset_id))

Each column in a **Dataset** is associated with a **Column Data Type**.
All we need to do now is to tell Abacus.AI what field we want to predict from our **Dataset**.

For each **Dataset Type** in a **Use Case**, there are special **Column Mappings** that can be applied to a column. We can find the list of available **Column Mappings** by calling the *Describe Use Case Requirements* API.

In [ ]:
client.describe_use_case_requirements(use_case)[0].allowed_column_mappings

We would map the *service* column as **SERVICE_ID**, the *LogCost* column as **VALUE**, and the *UsageStartDate* column as **DATE**.

The result of this call will be the updated schema after the **Column Mapping** is applied as stated above.

In [ ]:
client.set_column_mapping(project_id=cloud_project.project_id, dataset_id=cloud_dataset.dataset_id,
                          column='service', column_mapping='SERVICE_ID')

client.set_column_mapping(project_id=cloud_project.project_id, dataset_id=cloud_dataset.dataset_id,
                          column='LogCost', column_mapping='VALUE')

client.set_column_mapping(project_id=cloud_project.project_id, dataset_id=cloud_dataset.dataset_id,
                          column='UsageStartDate', column_mapping='DATE')

## 3. Train a Model

Now that we've uploaded the datasets, we're almost ready to train a model!

To be sure that our project is ready to go, let's call `project.validate` to confirm that all the project requirements have been met.

In [ ]:
cloud_project.validate()

For each **Use Case**, Abacus.AI has a bunch of options for training. We can call the *Get Training Config Options* API to see the available options.

In [ ]:
cloud_project.get_training_config_options()

In this notebook, we'll just train with the default options, but definitely feel free to experiment, especially if you have familiarity with Machine Learning.

In [ ]:
cloud_model = cloud_project.train_model(training_config={})
cloud_model.to_dict()

After we start training the model, we can call this blocking call that routinely checks the status of the model until it is trained and evaluated.

In [ ]:
cloud_model.wait_for_evaluation()

## **(Checkpoint)**
Training can take an hour or two to complete, but we encourage you to run the remaining calls on your own time. If your page times out or you hit refresh, you can restore your progress in this section.

In [ ]:
!pip install abacusai
import pandas as pd
import pprint
pp = pprint.PrettyPrinter(indent=2)
api_key = ''  #@param {type: "string"}
from abacusai import ApiClient
client = ApiClient(api_key)
cloud_project = next(project for project in client.list_projects() if project.name == 'Cloud Spend Project')
cloud_model = cloud_project.list_models()[-1]
cloud_model.wait_for_evaluation()

## Evaluate your Model Metrics

After your model is done training you can inspect the model's quality by reviewing the model's metrics:

In [ ]:
pp.pprint(cloud_model.get_metrics().to_dict())

To get a better understanding on what these metrics mean, visit our [documentation](https://abacus.ai/app/help/useCases/OPERATIONS_CLOUD/training) page.

## 4. Deploy Model

After the model has been trained, we need to deploy the model to be able to start making predictions. Deploying a model will reserve cloud resources to host the model for Realtime and/or batch predictions.

In [ ]:
cloud_deployment = cloud_model.create_deployment('Cloud Spend Deployment')
cloud_deployment.wait_for_deployment()

After the model is deployed, we need to create a deployment token for authenticating prediction requests. This token is only authorized to predict on deployments in this project, so it's safe to embed this token inside of a user-facing application or website.

In [ ]:
deployment_token = cloud_project.create_deployment_token().deployment_token
deployment_token

## 5. Predict


Now that you have an active deployment and a deployment token to authenticate requests, you can make the `get_anomalies` API call below.

This command will return information about a company's cloud spending, including the threshold, service, and anomalies. The anomaly detection would be performed using the data in the Cloud Spend dataset.

In [ ]:
ApiClient().get_anomalies(deployment_token=deployment_token, 
               deployment_id=cloud_deployment.deployment_id)